# Cargar datos y librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Basic
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statistics import mode
import os
import glob
import random
import cv2

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import cv2
# Features 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from skimage import feature
# Metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score #c alculate accuracy
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
# Classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm
#Save model
import joblib  

# Carga de datos 1

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/3-Caucasian Latin/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,168*192*2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2749


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/3-Caucasian Latin/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,168*192*2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


#Test

In [ ]:
#image = cv2.imread("/content/drive/MyDrive/Colab Notebooks/Base-De-Datos/datasetVGG/1-African American/train/african_001/0001_02.jpg", cv2.IMREAD_COLOR)

In [ ]:
#image = cv2.resize(image, (168,192))
#H,S,V = cv2.split(image)
#S.shape

In [ ]:
#image_merge = np.zeros((192,168,2))

#image_merge[:,:,0] = S
#image_merge[:,:,1] = V

In [ ]:
#print(V)

In [ ]:
#print(image_merge[:,:,1])

In [ ]:
#train_data, y_train = extract_data(path_train,"*.jpg",True)

In [ ]:
#test_data, y_test = extract_data(path_test,"*.jpg",True)

In [ ]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#y_train

# Separar datos en train y test

In [ ]:
#nsamples, nx, ny, chanels = train_data.shape

#train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

#train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
#nsamples, nx, ny, chanels = test_data.shape

#test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

#test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
#nsamples1, nx1, ny1, chanels = train_data.shape
#train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
#nsamples, nx, ny, chanels = test_data.shape
#test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255 

# PCA 

## Extracción de caracteristicas 

In [ ]:
n_components = 150

print("Extrayendo %d eigenfaces de %d caras"
      % (n_components, train_data.shape[0]))

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(train_data)


#eigenfaces = pca.components_.reshape((n_components,192, 162))

print("Projecting the input data on the eigenfaces orthonormal basis")

train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)

Extrayendo 150 eigenfaces de 2749 caras
Projecting the input data on the eigenfaces orthonormal basis


texto en negrita## SVM


## SVM

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

clf_pca_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_pca_svm = clf_pca_svm.fit(train_data_pca,y_train )

print("Best estimator found by grid search:")
print(clf_pca_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.005)


In [ ]:
y_pred = clf_pca_svm.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.7500    0.8400    0.7925        50
           2     0.6607    0.7400    0.6981        50
           3     0.5909    0.7800    0.6724        50
           4     0.9792    0.9400    0.9592        50
           5     0.9362    0.8800    0.9072        50
           6     0.7885    0.8200    0.8039        50
           7     0.9375    0.9000    0.9184        50
           8     0.9167    0.8800    0.8980        50
           9     0.9783    0.9000    0.9375        50
          10     0.8409    0.7400    0.7872        50
          11     0.9000    0.7200    0.8000        50
          12     0.8776    0.8600    0.8687        50
          13     0.6923    0.9000    0.7826        50
          14     0.8776    0.8600    0.8687        50
          15     0.9535    0.8200    0.8817        50
          16     0.7959    0.7800    0.7879        50
          17     0.8478    0.7800    0.8125        50
          18     0.8571    

In [ ]:
import pandas
df = pandas.DataFrame(report).transpose()
df.to_excel("svmPCA.xlsx")  

In [ ]:
def specificity(y_true,y_pred):
        cm=confusion_matrix(y_true, y_pred)
        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
        Specificity = TN/(TN+FP)    
        return np.mean(Specificity)
        
print(specificity(y_test, y_pred))

0.9915263157894738


## knn

In [ ]:
clf_PCA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_lbp_Kn_3 = clf_PCA_Knn_3.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_PCA_Knn_3.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.3820    0.6800    0.4892        50
           2     0.2597    0.4000    0.3150        50
           3     0.4474    0.3400    0.3864        50
           4     0.6897    0.8000    0.7407        50
           5     0.5278    0.3800    0.4419        50
           6     0.5741    0.6200    0.5962        50
           7     0.8636    0.7600    0.8085        50
           8     0.5000    0.4400    0.4681        50
           9     0.1959    0.9600    0.3254        50
          10     0.4681    0.4400    0.4536        50
          11     0.5417    0.2600    0.3514        50
          12     0.6000    0.3000    0.4000        50
          13     0.2812    0.3600    0.3158        50
          14     0.9167    0.2200    0.3548        50
          15     0.8421    0.3200    0.4638        50
          16     0.8421    0.3200    0.4638        50
          17     0.8667    0.2600    0.4000        50
          18     1.0000    

In [ ]:
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_pred))

0.43679747111395995


In [ ]:
print(specificity(y_test, y_pred))    

0.9712631578947368


## Gauss

In [ ]:
clf_PCA_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_PCA_Gauss.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_lbp_Gauss.predict(test_data_pca)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.7143    0.6000    0.6522        50
           2     0.4561    0.5200    0.4860        50
           3     0.2432    0.3600    0.2903        50
           4     0.9767    0.8400    0.9032        50
           5     0.8684    0.6600    0.7500        50
           6     0.6667    0.4400    0.5301        50
           7     0.7727    0.6800    0.7234        50
           8     0.7959    0.7800    0.7879        50
           9     0.3206    0.8400    0.4641        50
          10     0.7442    0.6400    0.6882        50
          11     0.9200    0.4600    0.6133        50
          12     0.9487    0.7400    0.8315        50
          13     0.5614    0.6400    0.5981        50
          14     0.8000    0.7200    0.7579        50
          15     0.9375    0.6000    0.7317        50
          16     0.7556    0.6800    0.7158        50
          17     0.6400    0.6400    0.6400        50
          18     0.6863    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.6284137863428524


In [ ]:
print(specificity(y_test, y_pred)) 

0.9812631578947366


# LDA

## Extracción de caracteristicas 

In [ ]:
lda = LDA(n_components=19)
X_train = lda.fit_transform(train_data, y_train)
train_data_LDA = lda.transform(train_data)
test_data_LDA = lda.transform(test_data)

## SVM

In [ ]:
clf_lda_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lda_svm = clf_lda_svm.fit(train_data_LDA,y_train )

print("Best estimator found by grid search:")
print(clf_lda_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.01)


In [ ]:
y_pred_lda_svm = clf_lda_svm.predict(test_data_LDA)
report = classification_report(y_test, y_pred_lda_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLDA.xlsx")  
print(classification_report(y_test, y_pred_lda_svm,digits=4))

              precision    recall  f1-score   support

           1     0.4535    0.7800    0.5735        50
           2     0.6383    0.6000    0.6186        50
           3     0.6970    0.4600    0.5542        50
           4     0.7797    0.9200    0.8440        50
           5     0.9394    0.6200    0.7470        50
           6     0.5098    0.5200    0.5149        50
           7     0.6800    0.6800    0.6800        50
           8     0.7917    0.7600    0.7755        50
           9     0.7143    0.9000    0.7965        50
          10     0.7292    0.7000    0.7143        50
          11     0.8529    0.5800    0.6905        50
          12     0.9737    0.7400    0.8409        50
          13     0.3491    0.7400    0.4744        50
          14     0.9286    0.7800    0.8478        50
          15     0.9500    0.7600    0.8444        50
          16     0.8684    0.6600    0.7500        50
          17     0.8462    0.6600    0.7416        50
          18     0.9118    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lda_svm))

0.6877055924336665


In [ ]:
print(specificity(y_test, y_pred_lda_svm)) 

0.9842631578947367


## knn

In [ ]:
clf_LDA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_LDA_Knn_3 = clf_PCA_Knn_3.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Knn_3.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("knnLDA.xlsx")  
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.6724    0.7800    0.7222        50
           2     0.5714    0.6400    0.6038        50
           3     0.6905    0.5800    0.6304        50
           4     0.8103    0.9400    0.8704        50
           5     0.7755    0.7600    0.7677        50
           6     0.5957    0.5600    0.5773        50
           7     0.8043    0.7400    0.7708        50
           8     0.6406    0.8200    0.7193        50
           9     0.7143    0.9000    0.7965        50
          10     0.6731    0.7000    0.6863        50
          11     0.7317    0.6000    0.6593        50
          12     0.9268    0.7600    0.8352        50
          13     0.5532    0.5200    0.5361        50
          14     0.9333    0.8400    0.8842        50
          15     0.9556    0.8600    0.9053        50
          16     0.8125    0.7800    0.7959        50
          17     0.8163    0.8000    0.8081        50
          18     0.7800    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.7361235170932198


In [ ]:
print(specificity(y_test, y_pred)) 

0.9867894736842105


## Gauss

In [ ]:
clf_LDA_Gauss = GaussianNB()

In [ ]:
clf_LDA_Gauss = clf_LDA_Gauss.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Gauss.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLDA.xlsx") 
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.6774    0.8400    0.7500        50
           2     0.5000    0.6600    0.5690        50
           3     0.8571    0.2400    0.3750        50
           4     0.9130    0.8400    0.8750        50
           5     1.0000    0.4600    0.6301        50
           6     0.3304    0.7600    0.4606        50
           7     0.8718    0.6800    0.7640        50
           8     0.8780    0.7200    0.7912        50
           9     0.3453    0.9600    0.5079        50
          10     0.8750    0.5600    0.6829        50
          11     0.9474    0.3600    0.5217        50
          12     1.0000    0.5600    0.7179        50
          13     0.3333    0.8600    0.4804        50
          14     0.9667    0.5800    0.7250        50
          15     1.0000    0.6400    0.7805        50
          16     0.9375    0.6000    0.7317        50
          17     0.9583    0.4600    0.6216        50
          18     0.9714    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.637926132092402


In [ ]:
print(specificity(y_test, y_pred)) 

0.9814736842105264


# Carga de datos 2

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/3-Caucasian Latin/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,192,168,2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2749


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/3-Caucasian Latin/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,192,168,2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


# Separar datos en train y test

In [ ]:
nsamples, nx, ny, chanels = train_data.shape

train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
nsamples, nx, ny, chanels = test_data.shape

test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
nsamples1, nx1, ny1, chanels = train_data.shape
train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
nsamples, nx, ny, chanels = test_data.shape
test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255

In [ ]:
train_data = train_data.reshape((nsamples1,nx1,ny1,chanels))
test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data_S = np.zeros((nsamples1,192,168))
train_data_S = train_data[:,:,:,0]
train_data_V = np.zeros((nsamples1,192,168))
train_data_V = train_data[:,:,:,1]

test_data_S = np.zeros((nsamples,192,168))
test_data_S = test_data[:,:,:,0]
test_data_V = np.zeros((nsamples1,192,168))
test_data_V = test_data[:,:,:,1] 

# LBP


## Extracción de caracteristicas

In [ ]:
# https://github.com/AsoStrife/Computer-Vision-Project

def blockshaped(arr, nrows, ncols):

    h, w = arr.shape
    return (arr.reshape(h//nrows, nrows, -1, ncols)
            .swapaxes(1,2)
            .reshape(-1, nrows, ncols))
def getHistogram(imgArray):
    hist, bin_edges = np.histogram(imgArray, density=True)
    return hist

In [ ]:
# tamaño de train y test
cont_train = train_data.shape[0]
cont_test = test_data.shape[0]
print(cont_test)

1000


In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_S = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_S[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_S[i] += x[0]

In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_V = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_V[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_V[i] += x[0]

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_S = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_S[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_S[i] += x[0]
    

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_V = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_V[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_V[i] += x[0]
    

In [ ]:
lbp_hist_test_V.shape

(1000, 1440)

In [ ]:
lbp_hist = np.concatenate((lbp_hist_S,lbp_hist_V),axis=1)
lbp_hist.shape

(2749, 2880)

In [ ]:
lbp_hist_test = np.concatenate((lbp_hist_test_S,lbp_hist_test_V),axis=1)
lbp_hist_test.shape

(1000, 2880)

## SMV

In [ ]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf_lbp_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lbp_svm = clf_lbp_svm.fit(lbp_hist,y_train )

print("El mejor conjunto de caracteriticas es: ")
print(clf_lbp_svm.best_estimator_)

El mejor conjunto de caracteriticas es: 
SVC(C=50000.0, class_weight='balanced', gamma=0.001)


In [ ]:
y_pred_lbp_svm = clf_lbp_svm.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_svm,digits=4))

              precision    recall  f1-score   support

           1     0.8723    0.8200    0.8454        50
           2     0.6667    0.7600    0.7103        50
           3     0.8400    0.8400    0.8400        50
           4     0.9608    0.9800    0.9703        50
           5     0.8750    0.8400    0.8571        50
           6     0.9286    0.7800    0.8478        50
           7     0.8364    0.9200    0.8762        50
           8     0.8571    0.8400    0.8485        50
           9     0.9375    0.9000    0.9184        50
          10     0.9074    0.9800    0.9423        50
          11     0.8000    0.8800    0.8381        50
          12     0.9388    0.9200    0.9293        50
          13     0.6897    0.8000    0.7407        50
          14     0.8936    0.8400    0.8660        50
          15     0.8462    0.8800    0.8627        50
          16     0.9512    0.7800    0.8571        50
          17     0.8837    0.7600    0.8172        50
          18     0.9038    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_svm))

0.8507143892332987


In [ ]:
print(specificity(y_test, y_pred_lbp_svm)) 

0.9925263157894735


## KNN

In [ ]:
clf_lbp_Kn_3 = KNeighborsClassifier(n_neighbors=7)

In [ ]:
clf_lbp_Kn_3 = clf_lbp_Kn_3.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_knn = clf_lbp_Kn_3.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_knn,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnLBP.xlsx")  
print(classification_report(y_test, y_pred_lbp_knn,digits=4))

              precision    recall  f1-score   support

           1     0.3585    0.7600    0.4872        50
           2     0.2889    0.5200    0.3714        50
           3     0.7333    0.2200    0.3385        50
           4     0.6104    0.9400    0.7402        50
           5     0.6129    0.7600    0.6786        50
           6     0.7917    0.3800    0.5135        50
           7     0.8684    0.6600    0.7500        50
           8     0.8000    0.4800    0.6000        50
           9     0.8261    0.7600    0.7917        50
          10     0.7500    0.6000    0.6667        50
          11     0.6346    0.6600    0.6471        50
          12     0.6818    0.9000    0.7759        50
          13     0.7778    0.2800    0.4118        50
          14     0.8261    0.3800    0.5205        50
          15     0.4658    0.6800    0.5528        50
          16     0.4407    0.5200    0.4771        50
          17     0.5246    0.6400    0.5766        50
          18     0.8889    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_knn))

0.5740344155317576


In [ ]:
print(specificity(y_test, y_pred_lbp_knn))  

0.9785263157894736


## Gauss

In [ ]:
clf_lbp_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_lbp_Gauss.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_gaus = clf_lbp_Gauss.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_gaus,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_gaus,digits=4))

              precision    recall  f1-score   support

           1     0.4627    0.6200    0.5299        50
           2     0.2500    0.0600    0.0968        50
           3     0.2576    0.3400    0.2931        50
           4     0.6531    0.6400    0.6465        50
           5     0.7879    0.5200    0.6265        50
           6     0.3208    0.3400    0.3301        50
           7     0.4737    0.5400    0.5047        50
           8     0.5682    0.5000    0.5319        50
           9     0.5641    0.4400    0.4944        50
          10     0.4051    0.6400    0.4961        50
          11     0.3523    0.6200    0.4493        50
          12     0.5833    0.7000    0.6364        50
          13     0.1471    0.1000    0.1190        50
          14     0.7097    0.4400    0.5432        50
          15     0.5625    0.5400    0.5510        50
          16     0.4138    0.4800    0.4444        50
          17     0.5625    0.1800    0.2727        50
          18     0.3824    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_gaus))

0.4269142396213767


In [ ]:
print(specificity(y_test, y_pred_lbp_gaus)) 

0.9712631578947368
